In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../src")

In [3]:
import numpy as np
import pandas as pd

# import fields
import model

In [4]:
sigma = model.MarginalParam("sigma", 1.0, (0.4, 3.5))
sigma.to_dataframe()

,name,value,bounds
0,sigma_11,1.0,"(0.4, 3.5)"
1,sigma_22,1.0,"(0.4, 3.5)"


In [5]:
params = model.MaternParams(2)

In [7]:
params.get_values()

AttributeError: 'str' object has no attribute 'to_dataframe'